In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [20]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv('Key_AzureOpenAI')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('Endpoint_AzureOpenAI')

os.environ['OPENAI_API_KEY'] = os.getenv('Key_OpenAI')

### Preload the vector store db

In [25]:
embeddings = OpenAIEmbeddings(
    openai_api_key=os.getenv('Key_OpenAI')
)

# def create_vector_database(txt_path):
#     loader=PyPDFDirectoryLoader(txt_path)
#     docs=loader.load()

#     documents = RecursiveCharacterTextSplitter(
#         chunk_size=1000, separators=["\n","\n\n"], chunk_overlap=200
#     ).split_documents(docs)
    
#     db = FAISS.from_documents(
#         documents=documents,
#         embedding=embeddings
#     )
#     db.save_local("../faiss-db")

# create_vector_database('../10k/')
vectorstore = FAISS.load_local("../faiss-db", embeddings, allow_dangerous_deserialization=True)
retriever_10k = vectorstore.as_retriever()

### Autogen Agents LLM

In [12]:
from autogen_core.components.models import AzureOpenAIChatCompletionClient, UserMessage

azure_autogen_llm = AzureOpenAIChatCompletionClient(
    model="gpt-4o",
    api_version="2024-02-01",
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True,
    }
)

### Tools

In [95]:
from autogen_core.components.tools import FunctionTool
import yfinance as yf
import pandas as pd

##################################################
# yahoo finance

def stock_prices(ticker: str) -> pd.DataFrame:
    """
    Get the historical prices and volume for a ticker for the last month.

    Args:
    ticker (str): the stock ticker to be given to yfinance

    """
    stock = yf.Ticker(ticker)
    df = stock.history()
    return df

def stock_news(ticker: str) -> list:
    """
    Get the most recent news of a stock or an instrument from Yahoo Finance

    Args:
    ticker (str): the stock ticker to be given to yfinance
    """
    stock = yf.Ticker(ticker)
    list_news = stock.news
    return list_news

def stock_shareholders(ticker: str) -> pd.DataFrame:
    """
    Get the major share holders info from Yahoo Finance
    """
    stock = yf.Ticker(ticker)
    df = stock.major_holders
    return df


def stock_incomestatement(ticker: str) -> pd.DataFrame:
    """
    Get the income statements info from Yahoo Finance
    """
    stock = yf.Ticker(ticker)
    df = stock.financials
    return df


##################################################
# RAG
def report_retriever(query: str) -> str:
    """
    Retrieve the relevant information
    """

    resources = retriever_10k.get_relevant_documents(query)

    text_result = ""
    for resource in resources:
        metadata = resource.metadata
        content = resource.page_content
        text = f"From document {metadata['source']} page {metadata['page']}, we have following information. \n {content} \n"
        text_result = text_result + text
    
    return text_result

###################################################
# UI


##############################################################################
# definiing the tools for the above functions
# in the future maybe able to pack this part in the form of decorator as CrewAI
stock_prices_tool = FunctionTool(stock_prices, description = 'Historical prices and volume for a ticker')
stock_news_tool = FunctionTool(stock_news, description = 'Most recent news of a stock or an instrument')
stock_shareholders_tool = FunctionTool(stock_shareholders, description = 'Shareholders of a stock or an instrument')
stock_incomestatement_tool = FunctionTool(stock_incomestatement, description = 'Income statement of a stock or an instrument')

report_retrieve_tool = FunctionTool(report_retriever, description= "Revlevant information from reports")

### Agents

In [96]:
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent

news_analysis_agent = ToolUseAssistantAgent(
    name="News_Analyzs_Agent",
    model_client=azure_autogen_llm,
    registered_tools=[stock_news],
    description="Get the most recent news of a stock or an instrument from Yahoo Finance",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

report_analysis_agent = ToolUseAssistantAgent(
    name="Report_Analyzs_Agent",
    model_client=azure_autogen_llm,
    registered_tools=[report_retriever],
    description="Get the relevant information from the report database",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

report_agent = CodingAssistantAgent(
    name="Report_Agent",
    model_client=azure_autogen_llm,
    description="Generate a report based on the search and reports analysis results, which is easy to read in python environment",
    system_message="You are a helpful assistant that can generate a comprehensive report on a given topic based on search and reports analysis results. When you done with generating the report, reply with TERMINATE.",
)

/var/folders/wp/6gm0gp7n5653wzt3d3527vsw0000gn/T/ipykernel_46654/1483858680.py:3: DeprecationWarning: ToolUseAssistantAgent is deprecated. Use AssistantAgent instead.
  news_analysis_agent = ToolUseAssistantAgent(
/var/folders/wp/6gm0gp7n5653wzt3d3527vsw0000gn/T/ipykernel_46654/1483858680.py:11: DeprecationWarning: ToolUseAssistantAgent is deprecated. Use AssistantAgent instead.
  report_analysis_agent = ToolUseAssistantAgent(
/var/folders/wp/6gm0gp7n5653wzt3d3527vsw0000gn/T/ipykernel_46654/1483858680.py:19: DeprecationWarning: CodingAssistantAgent is deprecated. Use AssistantAgent instead.
  report_agent = CodingAssistantAgent(


### Team

In [97]:
from autogen_agentchat.task import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat([news_analysis_agent, report_analysis_agent, report_agent], termination_condition=termination)

### Result

In [98]:
result = await team.run("Analysis the report and find relevant news to support it, finalize a summary of the report")

for message in result.messages:
    print(message.source + ":")
    print(message.content)
    print('________________________')

/Users/michael/PycharmProjects/2024BarclaysHackathonNew/.venv/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:299: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(
/Users/michael/PycharmProjects/2024BarclaysHackathonNew/.venv/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:299: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(
/Users/michael/PycharmProjects/2024BarclaysHackathonNew/.venv/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:299: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


user:
Analysis the report and find relevant news to support it, finalize a summary of the report
________________________
News_Analyzs_Agent:
Please provide me with the specific stock or instrument ticker symbol mentioned in the report for which you need relevant news.
________________________
Report_Analyzs_Agent:
Please provide the report details or the specific query related to the report so that I can retrieve the necessary information and summarize it for you.
________________________
Report_Agent:
Please upload or copy the report details here, and I will analyze it to find relevant news and finalize a summary for you.
________________________
News_Analyzs_Agent:
Please provide the details from the report, including the stock or instrument ticker symbol, any relevant data points, and key areas of focus. This information will help me find the latest news and provide a summary for you.
________________________
Report_Analyzs_Agent:
Please provide me with the query or details of the 

/Users/michael/PycharmProjects/2024BarclaysHackathonNew/.venv/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:258: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(
